In [2]:
import glob
import pandas as pd

from nltk.tokenize import TweetTokenizer, ToktokTokenizer
import re


In [14]:
pd.set_option('display.max_columns', len(df.columns))

In [5]:
random_flood = glob.glob('data/opendatascience Slack export Feb 8 2018/_random_flood/*.json')

In [269]:
deep_learning = glob.glob('data/opendatascience Slack export Feb 8 2018/deep_learning/*.json')

In [186]:
df = pd.concat(map(pd.read_json, deep_learning))

In [108]:
tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

In [254]:
def remove_links(text):
    return re.sub('<?\s?https?.+>?', '', text)

def substitute(text):
    text = re.sub('\:([_\w\d]+)\:', '_\\1_', text)
    text = re.sub('<[@#](\w)([\w\d]+)\|?[_\w\d]+>', '_\\1\\2_', text)
    return text

def filter_tokens(tokens):
    return len(tokens) >= 3

In [205]:
df['ts'] = df['ts'].astype(int).apply(pd.datetime.fromtimestamp)

In [237]:
df['period'] = df.ts.dt.round(pd.tseries.frequencies.Second(15))

In [238]:
texts = df[df.subtype.isnull()].groupby(['user', 'period'])['text'].apply('.'.join)

In [262]:
corpus = texts \
    .apply(lambda text: tokenizer.tokenize(substitute(text)))
    
corpus = corpus[corpus.apply(len) >= 3].apply(lambda tokens: remove_links(' '.join(tokens)))

In [267]:
corpus = corpus[corpus.str.len() > 10]
corpus = corpus.str.strip()

In [268]:
import csv
corpus.to_frame().to_csv('deep_learning.csv', sep='\t', index=False, header=False, quoting=csv.QUOTE_NONE, escapechar='\\')

In [251]:
corpus.size

31416